# Language Models for text generation
* One traditional approach
* And one deep approach

----

## The objective is to create a model which tries to assess the liklehood of language:

$P(w_{t+1} | w_{t-1+n}, ..., w_{t})$

---

## Traditional approach - trigram Markov Chain

* for a given dataset
* bin the words into size 2 unique pairs
* for a given pair, find the succeeding word
* build a transition probability matrix (markov chain) of these relationships
* use a sampler on this matrix to generate new text

---

### Imports

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import numpy as np
import re

### Download text data

In [ ]:
data = fetch_20newsgroups(remove=['headers', 'footers'])
text = data['data']

### Calculate the trigram probabilities

In [ ]:
def trigram_word_distribution(data):
    """create a probability distribution over all trigrams
    
        params: data - a Bunch data object from sklearn
        returns: [Bigram probability distribution, trigram probability distribution]
    """
    
    all_data = ' '.join([' '.join(re.findall('(?u)\\b\\w\\w*\\b',article.lower())) for article in text]).split()
    tri_gram = [' '.join([x,y]) for x,y in zip(all_data[:-1:], all_data[1::])]
    next_word = all_data[2:] + [' '] * 1
    words = pd.DataFrame({'seed_word':all_data[:-1],'gram_words':tri_gram, "next_word":next_word})
    words['seed_next_word'] = words['seed_word'].shift(-1)
    seed_word_distribution = words.groupby('seed_word')['seed_next_word'].value_counts(normalize=True)
    gram_word_distribution = words.groupby('gram_words')['next_word'].value_counts(normalize=True)
    
    return [seed_word_distribution, gram_word_distribution]

In [ ]:
dist = trigram_word_distribution(data)

### Generate a sentence

In [ ]:
def trigram_text_generation(seed, length, distribution):
    """seed a distribution with a seed word, and ask it to make more words
        
        params: seed - A seed word, 
                length -Length of the generated sentence
                distribution - A word probability distribution
                
        returns: generated sentence
    """
    
    try:
        seed = seed.lower()
        seed += ' ' + np.random.choice(distribution[0][seed].index, p=distribution[0][seed].values)
        for i in range(length):
             seed += ' ' + np.random.choice(distribution[1][' '.join(seed.split()[-2:])].index, p=distribution[1][' '.join(seed.split()[-2:])].values)
        return seed
    
    except:
        print('Oops! Try another word as seed word')
        return None

### An example

In [ ]:
seed = 'It'
sentence_length = 20
sentence = trigram_text_generation(seed,sentence_length, dist)
sentence

### Drawbacks:

* It is still imperfect at capturing anything above basic syntax, and has no semantics or pragmatic capability
* A quad-gram would be better, but as we increase gram size, transition matrices require increasingly more computation power to train, and space to store.
* N-grams are a sparse representation of language -  any word not present in the training corpus has a zero probability chance of being used

---

## Deep approach - Bidirectional LSTM with trainable Embeddings

### Imports

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, BatchNormalization, Flatten, Bidirectional, LSTM, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import re

### Create a continuous list of words

In [ ]:
all_data = ' '.join([' '.join(re.findall('(?u)\\b[a-zA-Z]*\\b',article.lower())) for article in text]).split()

### Work out the vocab list and size of vocab

In [ ]:
vocab_list = sorted(list(set(all_data)))
n_vocab = len(vocab_list)

### Create a mapping of words to numbers, to prepare input for the Embedding layer

In [ ]:
word_to_num = {}
num_to_word = {}
for i, word in enumerate(vocab_list):
    num_to_word[i] = word
    word_to_num[vocab_list[i]] = i

In [ ]:
word_to_num['because'], num_to_word[5593]

### Embed the data

In [ ]:
embedded_data = [word_to_num[word] for word in all_data]

### Create X and y, where y is the next word guess for X, the previous 10 words

In [ ]:
X_data= []
y_data = []
seq_length=10
for i in range(len(embedded_data)-seq_length):
    X_data.append(embedded_data[i:i+seq_length])
    y_data.append(embedded_data[i+seq_length])

### Reshape X and y data

In [ ]:
X = np.array(X_data).reshape(len(X_data), seq_length)
y = to_categorical(y_data) #onehot encode our y data

### Build the model

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=n_vocab, output_dim=32, input_length=seq_length))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128, return_sequences=True), merge_mode='sum'))
model.add(LSTM(128))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Save the best version of the model

In [ ]:
filepath=f"best_weights.hdf5"
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15, min_delta=0.0001) 
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
callbacks = [early_stop, checkpoint]

### Look at the parameters of the model

In [ ]:
model.summary()

### Fit the model

In [ ]:
history = model.fit(X, y, epochs=20, batch_size=512, validation_split=0.2, callbacks=callbacks)

### If trained, load weights from disk

In [ ]:
# load a saved model
# filename = "weights_01.hdf5"
# model.load_weights(filename)
# model.compile(loss='categorical_crossentropy', optimizer='adam')

----

### An example
* For a given input string generate some new text
* the input string has to be pre-prepared

In [ ]:
def prepare_input(seed_input):
    """prepare a string for the LSTM"""
    
    seed_input = seed_input.split()
    try:
        return np.expand_dims(np.array([word_to_num[x] for x in seed_input]).reshape(-1,1),axis=0)
    except:
        return 'please try with different words'

In [ ]:
def generate_text(input_string):
    """generate some new text as a string"""
    
    seed = prepare_input(input_string)
    for i in range(10):
    #predict next word based on window of 10 previous words - and add to embedded doc
        next_word = np.argmax(model.predict(seed[:,i:,:])).reshape(1,-1,1)
        seed = np.append(seed,next_word,axis=1)

    return ' '.join([num_to_word[x] for x in seed[0,:,0]])

### Unlike the traditional method, our seed has to be 10 words not 1 word

In [4]:
input_string = 'it was a dark and stormy night in berlin because'
assert len(input_string.split()) == 10

### Produce some text

In [ ]:
generate_text(input_string)

----